<a href="https://colab.research.google.com/github/yoo9519-AIdev/Learning/blob/master/Auto_mobile_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 라즈베리파이 코드입력방법(램1g / 메모리 60g)
1. yolov5로 차량 및 신호등 가중치를 학습시켜준 뒤 cvlib의 object_detection 코드 tuning  
(yoo9519_object_detection 리파짓 활용)
2. opencv 업데이트 후 라즈베리파이에 직접 코드입력(darknet clone하되, 가중치 학습 적용은 어떻게?)
3. 라즈베리파이 램 확장(최소 4g)
  
1번이 안되면 yolov3로 다시 학습시켜줘서 가중치를 구해준 뒤 적용해보기

### 방법 1

In [1]:
!curl -L "https://public.roboflow.com/ds/8nhfMDr86Z?key=WnpUWjqvrH" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 extracting: export/labels/1478900208139809310_jpg.rf.aa22f718963fd74816c36362747a2e5f.txt  
 extracting: export/labels/1478897095754019307_jpg.rf.aa1b8183dcaef29a615c9912089c3473.txt  
 extracting: export/labels/1478732017007188764_jpg.rf.a9fa40fbacf90200f65b9f36c6b43131.txt  
 extracting: export/labels/1478900676026204617_jpg.rf.aa1a8e8719bef56f09a0d6dd90f3f3ea.txt  
 extracting: export/labels/1478732798092393026_jpg.rf.aa228f5791738d87d1bacf62531c0cda.txt  
 extracting: export/labels/1478896275952367309_jpg.rf.aa23b59b1d8d54f3a09132da9e8c2a6c.txt  
 extracting: export/labels/1478899778529718267_jpg.rf.aa0d63ee3bcb35c1afed21e6540cdd76.txt  
 extracting: export/labels/1478901473546608423_jpg.rf.aa261619fcf6bfccf9da09aa1b0757fc.txt  
 extracting: export/labels/1478898797054295338_jpg.rf.aa06d409796399ae7edf8356b00c26a5.txt  
 extracting: export/labels/1478897897272970447_jpg.rf.aa398fcc4dbe3f9dd7487082af324c00.txt  
 extracting: export/labels/1478896

In [2]:
!git clone https://github.com/ultralytics/yolov3.git

Cloning into 'yolov3'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 9440 (delta 0), reused 2 (delta 0), pack-reused 9437
Receiving objects: 100% (9440/9440), 7.82 MiB | 9.11 MiB/s, done.
Resolving deltas: 100% (6466/6466), done.


In [3]:
%cd /content/yolov3/
!pip install -r requirements.txt

/content/yolov3
     |████████████████████████████████| 276kB 6.2MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44619 sha256=d91837dd875983be56d54bb16adfb1052c1edcc763e806101b79b4750734eb0f
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built PyYAML
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
%cat /content/yolov3_vehicle/data.yaml

train: ../train/images
val: ../valid/images

nc: 11
names: ['biker', 'car', 'pedestrian', 'trafficLight', 'trafficLight-Green', 'trafficLight-GreenLeft', 'trafficLight-Red', 'trafficLight-RedLeft', 'trafficLight-Yellow', 'trafficLight-YellowLeft', 'truck']

In [5]:
%cd /
from glob import glob

img_list = glob('/content/yolov3_vehicle/export/images/*.jpg')

print(len(img_list))

/
15000


In [6]:
from sklearn.model_selection import train_test_split

train_img_list, val_img_list = train_test_split(img_list, test_size=0.2, random_state=2000)

print(len(train_img_list), len(val_img_list))

12000 3000


In [7]:
with open('/content/yolov3_vehicle/train.txt', 'w') as f:
    f.write('\n'.join(train_img_list) + '\n')

with open('/content/yolov3_vehicle/val.txt', 'w') as f:
    f.write('\n'.join(val_img_list) + '\n')

In [8]:
import yaml

with open('/content/yolov3_vehicle/data.yaml', 'r') as f:
    data = yaml.load(f) # json 부르는 거랑 똑같이

print(data)

data['train'] = '/content/yolov3_vehicle/train.txt'
data['val'] = '/content/yolov3_vehicle/val.txt'

with open('/content/yolov3_vehicle/data.yaml', 'w') as f:
    yaml.dump(data, f)

print(data)

{'train': '../train/images', 'val': '../valid/images', 'nc': 11, 'names': ['biker', 'car', 'pedestrian', 'trafficLight', 'trafficLight-Green', 'trafficLight-GreenLeft', 'trafficLight-Red', 'trafficLight-RedLeft', 'trafficLight-Yellow', 'trafficLight-YellowLeft', 'truck']}
{'train': '/content/yolov3_vehicle/train.txt', 'val': '/content/yolov3_vehicle/val.txt', 'nc': 11, 'names': ['biker', 'car', 'pedestrian', 'trafficLight', 'trafficLight-Green', 'trafficLight-GreenLeft', 'trafficLight-Red', 'trafficLight-RedLeft', 'trafficLight-Yellow', 'trafficLight-YellowLeft', 'truck']}


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  after removing the cwd from sys.path.


In [9]:
!wget https://pjreddie.com/media/files/yolov3-tiny.weights

--2020-10-06 18:23:07--  https://pjreddie.com/media/files/yolov3-tiny.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35434956 (34M) [application/octet-stream]
Saving to: ‘yolov3-tiny.weights’

yolov3-tiny.weights 100%[===================>]  33.79M   149KB/s    in 3m 48s  

2020-10-06 18:26:57 (152 KB/s) - ‘yolov3-tiny.weights’ saved [35434956/35434956]



In [17]:
!git clone https://github.com/qqwweee/keras-yolo3.git

fatal: destination path 'keras-yolo3' already exists and is not an empty directory.


In [15]:
!python convert.py yolov3-tiny.cfg yolov3-tiny.weights model_data/yolo_tiny.h5

python3: can't open file 'convert.py': [Errno 2] No such file or directory


In [10]:
%cd /content/yolov3/

!python train.py --img 416 --batch 16 --epochs 75 --data /content/yolov3_vehicle/data.yaml --cfg /content/yolov3/cfg/yolov3-tiny.cfg --weights /content/yolov3/weights/yolov3-tiny.weights --name yolov3tiny_ADAS

/content/yolov3
Apex recommended for faster mixed precision training: https://github.com/NVIDIA/apex
Namespace(adam=False, batch_size=16, bucket='', cache_images=False, cfg='/content/yolov3/cfg/yolov3-tiny.cfg', data='/content/yolov3_vehicle/data.yaml', device='', epochs=75, evolve=False, freeze_layers=False, img_size=[416], multi_scale=False, name='yolov3tiny_ADAS', nosave=False, notest=False, rect=False, resume=False, single_cls=False, weights='/content/yolov3/weights/yolov3-tiny.weights')
Using CUDA device0 _CudaDeviceProperties(name='Tesla V100-SXM2-16GB', total_memory=16130MB)

Start Tensorboard with "tensorboard --logdir=runs", view at http://localhost:6006/
2020-10-06 18:37:23.846439: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Traceback (most recent call last):
  File "train.py", line 431, in <module>
    train(hyp)  # train normally
  File "train.py", line 80, in train
    data_dict = parse_data_cfg(data